In [1]:
import numpy as np
import pandas as pd
from config import *
import pickle as pkl
import os

In [2]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

/nethome/ss651/miniconda3/envs/sp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a Ber

In [22]:
files = [file for file in os.listdir("./raw_data/") if file.endswith(".csv")]

for file in files:
    data = pd.read_csv("./raw_data/"+file)
    text_column = dataset_category_mapping[file_name]["column_text"]
    data = data[data[text_column].apply(lambda x: isinstance(x, str))]
    print(len(data))
    file_name = file[:-4]
    text = data[text_column].tolist()
    labels = data[dataset_category_mapping[file_name]["column_name"]]
    label2id = dataset_category_mapping[file_name]["column_mapping"]
    encoded_text = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    output = model(**encoded_text)
    
    results = {}
    results['activations'] = output.pooler_output.detach().numpy()
    results['labels'] = [label2id[x] for x in labels.tolist()]
    
    pkl.dump(results, open("./BERTembeddings/"+file_name+".pkl", "wb"))


2999


In [21]:
for x in text:
    if not isinstance(x, str):
        print(x)
        print(type(x))

nan
<class 'float'>


In [16]:
import numpy as np
np.shape(output.pooler_output.detach().numpy())

(100, 768)